In [1]:
import pandas as pd
import numpy as np
import feedparser
from bs4 import BeautifulSoup
import requests
import PyPDF2 as pdf
import io
import os
import re
import weblist
import scrape_pdf

## Step 1: Scrape the list of webpages

In [2]:
mobile_url = "https://www.sam.usace.army.mil/Missions/Regulatory/Public-Notices/"
web_list = weblist.get_web_list(mobile_url)
web_list

,web_link,web_title,published_date,web_expire_date,pdf_url
0,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2023-00455-AMR PROPOSED DISCHARGE OF FILL...,9/6/2023,10/6/2023,https://www.sam.usace.army.mil//Portals/46/doc...
1,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2023-00077-MJF PROPOSED WETLAND FILL FOR ...,8/30/2023,9/29/2023,https://www.sam.usace.army.mil//Portals/46/doc...
2,http://www.sam.usace.army.mil/Missions/Regulat...,"SAM-2022-00542-CDJ PROPOSED NEW DREDGING, SHO...",8/30/2023,9/29/2023,https://www.sam.usace.army.mil//Portals/46/doc...
3,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2022-01041-JEB PROPOSED SHORELINE DEVELOP...,8/25/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/46/doc...
4,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2023-00405-STB PROPOSED RESIDENTIAL DEVEL...,8/24/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/46/doc...
...,...,...,...,...,...
893,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2012-01427-AFM PROPOSED WETLAND FILL IN KI...,11/26/2012,12/26/2012,https://www.sam.usace.army.mil//Portals/46/doc...
894,http://www.sam.usace.army.mil/Missions/Regulat...,MODIFICATION OF GENERAL PERMIT PROGRAM FOR MI...,11/20/2012,None,https://www.sam.usace.army.mil//Portals/46/doc...
895,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2012-00144-MBM PROPOSED BLACKWATER RIVER M...,11/16/2012,12/17/2012,https://www.sam.usace.army.mil//Portals/46/doc...
896,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2012-01418-DMY Ward Investment The Domain ...,11/16/2012,12/17/2012,https://www.sam.usace.army.mil//Portals/46/doc...


In [ ]:
web_list.to_csv("mobile_weblist.csv")

In [2]:
web_list = pd.read_csv("mobile_weblist.csv")
web_list_sub = web_list.loc[:99]

## Step2: Scrap PDFs

#### TEST

In [6]:
# single test
pdf_url = "https://www.sam.usace.army.mil/Portals/46/docs/regulatory/public_notices/SAM-2023-455-AMR.pdf?ver=BzL4PXrybMusB89PkkWgFw%3d%3d" 
pdf_text = scrape_pdf.pdf_read(pdf_url, "mobile")
pdf_text
# scrape_pdf.get_pdf_location(pdf_text, "mobile")
pdf_character = scrape_pdf.get_pdf_character(pdf_text, "mobile")
pdf_character
# scrape_pdf.get_pdf_acreage(pdf_character)
scrape_pdf.get_pdf_linear_feet(pdf_character)
# scrape_pdf.pdf_extraction(pdf_url, "mobile")
# scrape_pdf.get_pdf_mitigation(pdf_text, "mobile")

# char_sent_list = re.findall(r'.*?\D\.', pdf_character)
# char_sent_list[1]
# linear_feet_list = re.findall(r'([^,]*?\d*,?\d*\s?\.?\s?\d*-?\s?linear\sfeet.+?\D)(,|\.|\sand)', char_sent_list[1])
# linear_feet = re.search(r'[^A-Za-z]+(?=linear)', linear_feet_list[0][0]).group()
# for i in range(0, len(char_sent_list)):
#     if  any(w in char_sent_list[i] for w in ["impact", "affect", "loss"]):
#         linear_feet_list = re.findall(r'([^,]*?\d*,?\d*\s?\.?\s?\d*-?\s?linear\sfeet.+?\D)(,|\.|\sand)', char_sent_list[i])
#         linear_feet = re.search(r'[^A-Za-z]+(?=linear)', linear_feet_list[0][0]).group()
# linear_feet_list

# re.search(r'' + linear_feet + '.*?(of|to)(.*?(stream))', 
#                                                      linear_feet_list[0][0]).group(2).strip()
# re.search(r'(permanent[a-z]*\s|temporar[a-z]*\s)',
                                                     # linear_feet_list[0][0]).group().strip()

[('1104', 'intermittent stream', None)]

In [5]:
pdf_df = pd.DataFrame([scrape_pdf.pdf_extraction(x, "mobile") for x in web_list_sub["pdf_url"]],
                            columns = ['pdf_special',
                                       'pdf_comment_window',
                                       'pdf_permit_app_num',
                                       'pdf_dist_code',
                                       'pdf_dist_name',  
                                       'pdf_manager_name', 
                                       'pdf_manager_phone', 
                                       'pdf_manager_email',
                                       'pdf_applicant_contents',
                                       'pdf_applicant',
                                       'pdf_contract',
                                       'pdf_location', 
                                       'pdf_character', 
                                       'pdf_mitigation',
                                       'pdf_county',
                                       'pdf_parish',
                                       'pdf_city',
                                       'pdf_hydrologic_unit_code', 
                                       'pdf_longitude',
                                       'pdf_latitude',
                                       'pdf_wqc',
                                       'pdf_cup',
                                       'pdf_acre1',
                                       'pdf_acre_type1', 
                                       'pdf_acre_term1',
                                       'pdf_acre2',
                                       'pdf_acre_type2', 
                                       'pdf_acre_term2', 
                                       'pdf_acre3',
                                       'pdf_acre_type3', 
                                       'pdf_acre_term3',
                                       'pdf_acre4',
                                       'pdf_acre_type4', 
                                       'pdf_acre_term4',
                                       'pdf_acre5',
                                       'pdf_acre_type5', 
                                       'pdf_acre_term5', 
                                       'pdf_acre6',
                                       'pdf_acre_type6', 
                                       'pdf_acre_term6', 
                                       'pdf_lf1', 
                                       'pdf_lf_type1', 
                                       'pdf_ldf_term1', 
                                       'pdf_lf2', 
                                       'pdf_lf_type2', 
                                       'pdf_ldf_term2', 
                                       'pdf_lf3', 
                                       'pdf_lf_type3', 
                                       'pdf_ldf_term3', 
                                       'pdf_text'])

# # Merge with RSS feed table
final_df = web_list_sub.join(pdf_df)

# Add a new column to track the number of errors for each webpage
final_df["error"] = final_df.apply(lambda row: row.str.contains("ERROR").sum(), axis = 1)

final_df

,Unnamed: 0,web_link,web_title,published_date,web_expire_date,pdf_url,pdf_special,pdf_comment_window,pdf_permit_app_num,pdf_dist_code,...,pdf_lf_type1,pdf_ldf_term1,pdf_lf2,pdf_lf_type2,pdf_ldf_term2,pdf_lf3,pdf_lf_type3,pdf_ldf_term3,pdf_text,error
0,0,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2023-00455-AMR PROPOSED DISCHARGE OF FILL...,9/6/2023,10/6/2023,https://www.sam.usace.army.mil//Portals/46/doc...,0,30,SAM-2023-00455-AMR,SAM,...,intermittent stream,None,None,None,None,None,None,None,DEPARTMENT OF THE ARM Y U.S. ARMY CORPS OF ENG...,0
1,1,http://www.sam.usace.army.mil/Missions/Regulat...,SAM-2023-00077-MJF PROPOSED WETLAND FILL FOR ...,8/30/2023,9/29/2023,https://www.sam.usace.army.mil//Portals/46/doc...,0,30,SAM-2023-00077-MJF,SAM,...,a culvert along 60- linear feet of stream,None,None,None,None,None,None,None,DEPARTMENT OF THE ARMY U.S. ARMY CORPS OF ENG...,0
2,2,http://www.sam.usace.army.mil/Missions/Regulat...,"SAM-2022-00542-CDJ PROPOSED NEW DREDGING, SHO...",8/30/2023,9/29/2023,https://www.sam.usace.army.mil//Portals/46/doc...,0,30,SAM-2022-00542-CDJ,SAM,...,None,None,None,None,None,None,None,None,DEPARTMENT OF THE ARMY U.S. ARMY CORPS OF ENG...,0


In [4]:
final_df.to_csv("mobile_subset.csv")